# **ML Final Project**:

### **Group Members:**
- Christopher Johnson (christopher.johnson13@ontariotechu.net)
- Name (Student Email)
- Name (Student Email)
- Name (Student Email)

## **Project Goals & Outline:**
The goal of this project is to use sentiment analysis to analyze the content of tweets, and make decisions on whether their sentiment is positive, negative, neutral, or irrelevant.

### Outline:
1. Data Importing and Preprocessing
2. Model Construction
   1. RNN
   2. LSTM
   3. ???
3. Model Training
   1. RNN
   2. LSTM
   3. ???
4. Model Analysis and Comparison
5. Conclusions

## **Importing Packages & Libraries:**

In [27]:
# general packages/libraries
import numpy as np
import pandas as pd
from datetime import datetime # used to convert Date_time strings to useable format

# torch
import torch

# torchmetrics
from torchmetrics import Accuracy

# torchtext
import torchtext.data
from torchtext.vocab import build_vocab_from_iterator

# lightning
from lightning.pytorch import LightningModule
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import CSVLogger

# tqdm
from tqdm.notebook import tqdm

## **Data Importing and Preprocessing:**
This section is where the tweet data is imported and processed into tokens. This tokenization process is required so the neural network architectures can interpret the text data.

**Tokenization definition:** the process of breaking down a sequence of information into smaller chunks known as tokens.

### Importing the Dataset:

In [20]:
# import the tweet data from the CSV using pandas
tweet_data = pd.read_csv('./Datasets/twitter_training.csv', names=['Tweet ID', 'entity', 'sentiment', 'Tweet content'])
tweet_data.head(20)

,Tweet ID,entity,sentiment,Tweet content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
5,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
6,2402,Borderlands,Positive,So I spent a few hours making something for fu...
7,2402,Borderlands,Positive,So I spent a couple of hours doing something f...
8,2402,Borderlands,Positive,So I spent a few hours doing something for fun...
9,2402,Borderlands,Positive,So I spent a few hours making something for fu...


In [34]:
# extracting the the tweet content for ease of use
content = tweet_data['Tweet content']

# TODO: (if necessary) preprocess the content
# e.g. remove any nonsense or very short tweets

# showing the extracted content
print(content.head(20))

# convert content to an iterator
content = iter(content)

0     im getting on borderlands and i will murder yo...
1     I am coming to the borders and I will kill you...
2     im getting on borderlands and i will kill you ...
3     im coming on borderlands and i will murder you...
4     im getting on borderlands 2 and i will murder ...
5     im getting into borderlands and i can murder y...
6     So I spent a few hours making something for fu...
7     So I spent a couple of hours doing something f...
8     So I spent a few hours doing something for fun...
9     So I spent a few hours making something for fu...
10    2010 So I spent a few hours making something f...
11                                                  was
12    Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...
13    Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...
14    Rock-Hard La Varlope, RARE & POWERFUL, HANDSOM...
15    Rock-Hard La Vita, RARE BUT POWERFUL, HANDSOME...
16    Live Rock - Hard music La la Varlope, RARE & t...
17    I-Hard like me, RARE LONDON DE, HANDSOME 2

In [35]:
# extracting the the sentiment data for ease of use
sentiment = tweet_data['sentiment']

# the numerical representations of the sentiment values
sentiment_numerical = {
    'Positive': 0,
    'Negative': 1,
    'Neutral': 2,
    'Irrelevant': 3
}

# converting the sentiment data into a numerical form
sentiment.replace(to_replace=sentiment_numerical, inplace=True)

# showing the converted sentiment data
print(sentiment.head(20))

# converting the sentiment data into a torch tensor
sentiment = torch.tensor(sentiment, dtype=torch.int64)

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    2
13    2
14    2
15    2
16    2
17    2
18    0
19    0
Name: sentiment, dtype: int64


### Tokenization:

In [45]:
# TODO: Fix tokenizer

# tokenizer = torchtext.data.get_tokenizer('basic_english')

# # creates tokens from a given dataframe
# def create_tokens(values):
#     for val in tqdm(values):
#         print(val)
#         yield tokenizer(val)

# # create the vocab from the content data
# vocab = build_vocab_from_iterator(
#     create_tokens(content),
#     min_freq=5,
#     specials=['<unk>', '<s>', '<eos>']
# )
# vocab.set_default_index(0)

## **Model Creation:**
This is where the models being used are defined.

### **RNN Model:**